In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score
from tqdm import tqdm
import re
import unicodedata

### Доп функции предобработки:

In [ ]:
def clean_text(text):
    """Очистка текста"""

    # Проверка на NaN/None
    if not isinstance(text, str):
        return ""

    # 1. Нижний регистр
    text = text.lower()

    # 2. Удаление эмодзи и спец символов (оставляем буквы, цифры, пунктуацию, пробелы)
    cleaned = []
    for char in text:
        category = unicodedata.category(char)
        if category[0] in ['L', 'N', 'P', 'Z']:
            cleaned.append(char)
    text = ''.join(cleaned)

    # 3. Защита многоточия
    text = text.replace('...', '__ELLIPSIS__')

    # 4. Уменьшение повторяющихся знаков до одного
    text = re.sub(r'([!?;:,\-—–])\1+', r'\1', text)

    # 5. Восстановление многоточия
    text = text.replace('__ELLIPSIS__', '...')

    # 6. Удаление лишних пробелов
    text = re.sub(r'\s{2,}', ' ', text)

    # 7. Удаление пробелов в начале/конце
    text = text.strip()

    return text


In [ ]:
def combine_text_with_src(df, text_col='text', src_col='src', output_col='text'):
    """
    Склеивает текст из двух столбцов по шаблону:
    [SRC] текст из source [TEXT] текст из text

    Args:
        df (pd.DataFrame): Исходный датафрейм
        text_col (str): Имя столбца с текстом
        src_col (str): Имя столбца с источником
        output_col (str): Имя выходного столбца

    Returns:
        pd.DataFrame: DataFrame с новым столбцом
    """

    # Проверка наличия столбцов
    if text_col not in df.columns:
        raise ValueError(f"Столбец '{text_col}' не найден")
    if src_col not in df.columns:
        raise ValueError(f"Столбец '{src_col}' не найден")

    # Склеиваем строки
    df[output_col] = "[SRC] " + df[src_col].astype(str) + " [TEXT] " + df[text_col].astype(str)

    return df


In [ ]:
def print_token_stats(lengths):
    print(f"Минимальное количество токенов: {np.min(lengths)}")
    print(f"Максимальное количество токенов: {np.max(lengths)}")
    print(f"Среднее количество токенов: {np.mean(lengths):.2f}")
    print(f"Медиана количества токенов: {np.median(lengths):.2f}")
    print(f"Стандартное отклонение: {np.std(lengths):.2f}")
    print(f"25-й перцентиль: {np.percentile(lengths, 25):.2f}")
    print(f"75-й перцентиль: {np.percentile(lengths, 75):.2f}")
    print(f"Общее количество комментариев: {len(lengths)}")


### Основной пайплайн

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/models/cointegrated_ART"

BASE_MODEL_NAME = "ai-forever/ruBert-base"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Inference device: {device}")

Inference device: cuda


In [ ]:
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    print("Tokenizer loaded from local folder.")
except Exception as e:
    print(f"Warning: Local tokenizer failed ({e}). Loading from HuggingFace Hub")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

Tokenizer loaded from local folder.


In [ ]:
try:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
    model.to(device)
    model.eval()
    print("Model loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Could not load model from {MODEL_DIR}. Error: {e}")

Model loaded successfully.


In [ ]:
id2label = {0: "neutral", 1: "positive", 2: "negative"}

In [ ]:
def process_and_predict(input_csv_path: str, output_csv_path: str):
    """
    Принимает путь к входному CSV, делает предсказания, сохраняет результат.
    Возвращает: (metric_f1, output_path)
    """

    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"Input file not found: {input_csv_path}")

    try:
        df = pd.read_csv(input_csv_path)
    except:
        df = pd.read_csv(input_csv_path, sep=';')

    if 'text' not in df.columns:
        raise ValueError("CSV must contain a 'text' column!")

    # Очистка от NaN
    df['text'] = df['text'].fillna("")
    df['text'] = df['text'].apply(clean_text)

    if 'src' in df.columns:
        df = combine_text_with_src(df, text_col='text', src_col='src', output_col='text')
        inference_column = 'text'
    else:
        inference_column = 'text'

    # Берем готовый список текстов
    texts = df[inference_column].tolist()
    # Инференс (батчами)
    batch_size = 32
    predictions = []

    print(f"Predicting for {len(texts)} texts")

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i : i + batch_size]

        # Токенизация
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        ).to(device)

        # Предикт
        with torch.no_grad():
            logits = model(**inputs).logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

        predictions.extend(preds)


    df['predicted_label'] = predictions


    # Считаем метрику (если есть истинные метки)
    f1_macro = None
    if 'label' in df.columns:
        try:
            true_labels = df['label'].astype(int).tolist()
            f1_macro = f1_score(true_labels, predictions, average='macro')
            print(f"Validation Macro-F1: {f1_macro:.4f}")
        except Exception as e:
            print(f"Metric calculation skipped: {e}")

    # Сохраняем
    # Создаем папку для вывода, если нет
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

    df.to_csv(output_csv_path, index=False)
    print(f"Saved results to: {output_csv_path}")

    return f1_macro, output_csv_path

In [ ]:
if __name__ == "__main__":
    TEST_FILE = "/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/raw/test.csv"
    RESULT_FILE = "/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/processed/new_test_predictions.csv"

    if os.path.exists(TEST_FILE):
        metric, out_path = process_and_predict(TEST_FILE, RESULT_FILE)
        print(f"Result saved to: {out_path}")
        print(f"Metric: {metric}")
    else:
        print(f"Test file '{TEST_FILE}' not found. Skipping check.")

Predicting for 58092 texts


100%|██████████| 1816/1816 [01:23<00:00, 21.71it/s]


Saved results to: /content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/processed/new_test_predictions.csv
Result saved to: /content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/processed/new_test_predictions.csv
Metric: None
